In [1]:
from keras.datasets import mnist
import numpy as np
from matplotlib import pyplot as plt

Using TensorFlow backend.


In [2]:
import tensorflow as tf
from tensorflow.contrib import layers
from PIL import Image

In [3]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [ ]:
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')

# Scaling the range of the image to [-1, 1]
# Because we are using tanh as the activation function in the last layer of the generator
# and tanh restricts the weights in the range [-1, 1]
X_train = (X_train - 127.5) / 127.5

X_train.shape

In [ ]:
def weight_variable(shape):
    """
    Create a weight variable with appropriate initialization
    :param name: weight name
    :param shape: weight shape
    :return: initialized weight variable
    """
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('W', dtype=tf.float32, shape=shape, initializer=initer)

In [ ]:
def bias_variable(shape):
    """
    Create a bias variable with appropriate initialization
    :param name: bias variable name
    :param shape: bias variable shape
    :return: initialized bias variable
    """
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('b', dtype=tf.float32, initializer=initial)

In [ ]:
def fc_layer(x, num_units, name, use_relu=True):
    """
    Create a fully-connected layer
    :param x: input from previous layer
    :param num_out: number of hidden units in the fully-connected layer
    :param name: layer name
    :param use_relu: boolean to add ReLU non-linearity (or not)
    :return: The output array
    """
    with tf.variable_scope(name):
        in_dim = x.get_shape()[1]
        W = weight_variable(shape=[in_dim, num_units])
        tf.summary.histogram('weight', W)
        b = bias_variable(shape=[num_units])
        tf.summary.histogram('bias', b)
        layer = tf.matmul(x, W)
        layer += b
        if use_relu:
            layer = tf.nn.relu(layer)
        return layer

### Using PRelu (parametarized leaky relu to correctly avoid dying Relu problem )  and Leaky Relu problem where negetive data only provide small amount of slope for learning parameter so that network itself familirize the required slope paramter for negative input value

In [ ]:
def Prelu(inpt, name):
    """
    @inpt: input from previous layer
    @name: layer name
    """
    with tf.variable_scope(name):
        i = int(inpt.get_shape()[-1])
        alpha = tf.get_variable('alpha', shape=(i,))
        output = tf.nn.relu(inpt) + tf.multiply(alpha, -tf.nn.relu(-inpt))
    return output

In [ ]:
def conv_layer(x, num_filters, filter_size, stride, name, use_relu=True):
    """
    Create a 2D convolution layer
    :param x: input from previous layer
    :param filter_size: size of each filter
    :param num_filters: number of filters (or output feature maps)
    :param stride: filter stride
    :param name: layer name
    :return: The output array
    """
    with tf.variable_scope(name):
        num_in_channel = x.get_shape().as_list()[-1]
        shape = [filter_size, filter_size, num_in_channel, num_filters]
        W = weight_variable(shape=shape)
        tf.summary.histogram('weight', W)
        b = bias_variable(shape=[num_filters])
        tf.summary.histogram('bias', b)
        layer = tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding="SAME")
        layer += b
        if use_relu:
#         Using non - linearity RELU
            return tf.nn.relu(layer)
        return layer

In [ ]:
def flatten_layer(layer):
    """
    Flattens the output of the convolutional layer to be fed into fully-connected layer
    :param layer: input array
    :return: flattened array
    """
    with tf.variable_scope('Flatten_layer'):
        layer_shape = layer.get_shape()
        num_features = layer_shape[1:4].num_elements()
        layer_flat = tf.reshape(layer, [-1, num_features])
    return layer_flat

In [ ]:
def dropout(x, keep_prob,name):
    """Create a dropout layer."""
    return tf.nn.dropout(x, keep_prob,name=name)

In [ ]:
lr = 0.001  # The optimization initial learning rate
epochs = 20  # Total number of training epochs
batch_size = 50  # Training batch size
display_freq = 50  # Frequency of displaying the training results

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1], name='X')
y = tf.placeholder(tf.float32, shape=[None, 1], name='Y')
noise = tf.placeholder(tf.float32,shape=[None, 64])
phase = tf.placeholder(tf.bool, name='phase')

In [ ]:
# Generator network
g_net = fc_layer(noise,1024,name='g_fc1',use_relu=False)
g_net = Prelu(g_net, name='g_prelu_1')
#     Post activation Batch Normalization 
g_net = layers.batch_norm(g_net, is_training=phase)

g_net = fc_layer(g_net,7 * 7 * 256, name='g_fc2', use_relu=False)
g_net = Prelu(g_net, name='g_prelu_2')
#     Post activation Batch Normalization
g_net = layers.batch_norm(g_net, is_training=phase)

g_net = tf.reshape(g_net, [-1, 7, 7, 256])
g_net = layers.conv2d_transpose(g_net, 64, [4, 4], stride=2)
g_net = layers.conv2d_transpose(g_net, 32, [4, 4], stride=2)
# Make sure that generator output is in the same range as `inputs`
# ie [-1, 1].
g_output = layers.conv2d(g_net, 1, 4, normalizer_fn=None, activation_fn=tf.tanh)

In [ ]:
# Discriminator network
d_net = conv_layer(x, 64, 5, stride = 2, name='d_conv_1',use_relu=False)
d_net = Prelu(d_net, name='d_prelu_1')

d_net = dropout(d_net, 0.3)

d_net = conv_layer(d_net, 128, 5, stride = 2, name='d_conv_2',use_relu=False)
d_net = Prelu(d_net, name='d_prelu_2')

d_net = dropout(d_net, 0.3)

d_net = flatten_layer(d_net)
d_net = fc_layer(d_net,1,name='output_discriminator',use_relu=False)
d_output= tf.nn.sigmoid(d_net)

In [ ]:
with tf.variable_scope('generator'):
    with tf.variable_scope('Train'):
        with tf.variable_scope('Loss'):
            g_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=g_output), name='loss')
        tf.summary.scalar('loss', g_loss)
        with tf.variable_scope('Optimizer'):
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops):
                g_optimizer = tf.train.AdamOptimizer(learning_rate=0.0001, name='Adam-op').minimize(g_loss)
        with tf.variable_scope('Accuracy'):
            g_correct_prediction = tf.equal(tf.argmax(g_output, 1), tf.argmax(y, 1), name='correct_pred')
            g_accuracy = tf.reduce_mean(tf.cast(g_correct_prediction, tf.float32), name='accuracy')
        tf.summary.scalar('accuracy', accuracy)
        with tf.variable_scope('Prediction'):
            g_cls_prediction = tf.argmax(g_output, axis=1, name='predictions')

In [ ]:
with tf.variable_scope('discriminator'):
    with tf.variable_scope('Train'):
        with tf.variable_scope('Loss'):
            d_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=d_output), name='loss')
        tf.summary.scalar('loss', d_loss)
        with tf.variable_scope('Optimizer'):
            d_optimizer = tf.train.AdamOptimizer(learning_rate=0.0001, name='Adam-op').minimize(d_loss)
        with tf.variable_scope('Accuracy'):
            d_correct_prediction = tf.equal(tf.argmax(output_logits, 1), tf.argmax(y, 1), name='correct_pred')
            d_accuracy = tf.reduce_mean(tf.cast(d_correct_prediction, tf.float32), name='accuracy')
        tf.summary.scalar('accuracy', accuracy)
        with tf.variable_scope('Prediction'):
            d_cls_prediction = tf.argmax(output_logits, axis=1, name='predictions')